### Final notebook

In [ ]:
%pylab inline
import desispec
import desitarget
import desimodel.io
import pickle
import fitsio
from astropy.table import Table
from astropy.io import fits

In [ ]:
from qso_templates import qsoproc
from qso_templates import outlier_detection
from qso_templates import run_hmf
from qso_templates import empca

In [ ]:
rest_offset = 2.6
nbins = 13500
min_per_lam=50

### Pre-process spectra ###

In [ ]:
%%time
fluxes, ivars, qsocat = qsoproc.efficient_pipeline(100, nkeep=2, rlb=rest_offset, nbins=nbins)

In [ ]:
fluxes.shape

In [ ]:
len(qsocat)

In [ ]:
import pickle
with open("data/final_data2/fluxes2.pkl","wb") as file:
    pickle.dump(fluxes,file)
    
with open("data/final_data2/ivars2.pkl", "wb") as file:
    pickle.dump(ivars, file)
    
qsocat.write('data/final_data2/qsocat_selected2.fits', format='fits', overwrite=True)

In [ ]:
imshow(fluxes, aspect='auto', vmin=0, vmax=2);

### Quality cuts ###

In [ ]:
with open('data/final_data2/fluxes2.pkl',"rb") as file:
    fluxes = pickle.load(file)

with open('data/final_data2/ivars2.pkl',"rb") as file:
    ivars = pickle.load(file)

qsocat = Table(fits.getdata('data/final_data2/qsocat_selected2.fits'))

In [ ]:
#- Removes wavelengths with < min_spectra nonzero observations
fluxes, ivars = outlier_detection.enough_obs(fluxes, ivars, min_spectra=min_per_lam)
print(fluxes.shape)
imshow(ivars, aspect='auto', vmin=0, vmax=2);

In [ ]:
old_dist = outlier_detection.calc_chisq_dist(fluxes, ivars)
outlier_detection.plot_chisq_dist(old_dist)

In [ ]:
plot_chisq_dist(old_dist, hist=True)

In [ ]:
new_f, new_i, new_dist, qsocat = outlier_detection.outlier_detection(fluxes, ivars, qsocat)

In [ ]:
outlier_detection.plot_chisq_dist(new_dist)

In [ ]:
outlier_detection.plot_chisq_dist(new_dist, hist=True)

In [ ]:
print(new_f.shape)

In [ ]:
with open("data/final_data2/fluxes2_good.pkl","wb") as file:
    pickle.dump(new_f,file)
with open("data/final_data2/ivars2_good.pkl","wb") as file:
    pickle.dump(new_i,file)
qsocat.write('data/final_data2/qsocat_selected2_good.fits', format='fits', overwrite=True)

### HMF-factorization ###

In [ ]:
with open("data/final_data2/fluxes2_good.pkl","rb") as file:
    fluxes = pickle.load(file)
    
with open("data/final_data2/ivars2_good.pkl","rb") as file:
    ivars = pickle.load(file)

fluxes.shape

In [ ]:
f2, i2 = run_hmf.enough_obs(fluxes, ivars, min_per_lam)
f2.shape

In [ ]:
with open("data/final_data2/final_fluxes.pkl","wb") as file:
    pickle.dump(f2, file)
    
with open("data/final_data2/final_ivars.pkl","wb") as file:
    pickle.dump(i2, file)

In [ ]:
imshow(i2, aspect='auto', vmin=0, vmax=2);

In [ ]:
# Get and store loglams
bin_range = np.arange(f2.shape[1])
rest_loglam_diffs = bin_range * 0.0001
rest_loglams = np.array(rest_loglam_diffs + rest_offset)
rest_loglams

In [ ]:
rest_loglams.shape

In [ ]:
wavelengths = np.power(10, rest_loglams)
wavelengths

In [ ]:
with open("data/final_data2/final_rest_loglams.pkl","wb") as file:
    pickle.dump(rest_loglams, file)

In [ ]:
%%time
V, C, chistats = run_hmf.hmf_weighted(fluxes, ivars, 5, num_iter=5,)

with open("data/final_data2/model_V.pkl","wb") as file:
    pickle.dump(V,file)
    
with open("data/final_data2/model_C.pkl", "wb") as file:
    pickle.dump(C, file)

In [ ]:
with open("data/final_data2/model_V.pkl","rb") as file:
    V = pickle.load(file)
    
with open("data/final_data2/model_C.pkl", "rb") as file:
    C = pickle.load(file)

In [ ]:
V.shape

In [ ]:
C.shape

In [ ]:
model_hmf = np.dot(V, C).T

In [ ]:
model_hmf.shape

In [ ]:
with open("data/final_data2/model_hmf.pkl","wb") as file:
    pickle.dump(model_hmf,file)

### EMPCA 

In [ ]:
# Extract model from file
with open("data/final_data2/model_hmf.pkl","rb") as file:
    model_hmf = pickle.load(file)

In [ ]:
# create uniform weights to go along with it
weights = np.ones(model_hmf.shape)

In [ ]:
plot(np.arange(model_hmf.shape[1]), np.average(model_hmf, axis=0, weights=weights));

In [ ]:
final_model = empca(data=model_hmf, weights=weights)

In [ ]:
with open("data/final_data2/final_model.pkl","wb") as file:
    pickle.dump(final_model, file)

In [ ]:
with open("data/final_data2/rest_loglams.pkl","rb") as file:
    rest_loglams = pickle.load(file)
    
wavelengths = 10**rest_loglams

**These are the plots of all the eigenvectors:**

In [ ]:
plot(wavelengths, final_model.eigvec[0]);
xlim(4500, 5500);

In [ ]:
plot(wavelengths, final_model.eigvec[1]);

In [ ]:
plot(wavelengths, final_model.eigvec[2]);

In [ ]:
plot(wavelengths, final_model.eigvec[3]);

In [ ]:
plot(wavelengths, final_model.eigvec[4]);

In [ ]:
i=0
plot(model_hmf.data[i], alpha=0.6);
plot(final_model.model[i])